In [1]:
import sys

import os
import json
import time
import argparse

import pandas as pd
import numpy as np


import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, Subset

from torchinfo import summary

from sklearn.metrics import f1_score, roc_auc_score, roc_curve, auc, precision_recall_curve
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler

import wandb

sys.path.append('./../../../src/')

from utils import *
from utils_torch import * 
from MHCCBM import *
from PGPredictor_CNN import *
from tqdm import tqdm

In [4]:
MAPP_df = pd.read_csv('./../../../data/PG/MAPP_AP_benchmark.csv', sep='\t')
MAPP_df['y'] = [1 if i=='HIT' else 0 for i in MAPP_df['HIT/DECOY']]
MAPP_df

peptide       Gene n_flank c_flank  length  AP with flanks score  \
0       FKDTDYKRH      NT5C2   RTSVD   KDTDY       9              0.084120   
1        TQIMFETF       ACTB   NREKM   QIMFE       8              0.377213   
2     KKSQIFSTASD      HSPA5   TVVPT   KSQIF      11              0.014379   
3       KTNHLVTVE       PDHB   EASVM   TNHLV       9              0.120193   
4     SDLQLDRISVY       TUBB   TYHGD   DLQLD      11              0.089185   
...           ...        ...     ...     ...     ...                   ...   
3169  VRGLLAGRVPP      GPAT2   VGACA   RGLLA      11              0.412522   
3170  AQKKDGKKRKR  HIST1H2BI   KAVTK   QKKDG      11              0.065142   
3171    LKQKRMYEQ      CHMP5   KALRV   KQKRM       9              0.072208   
3172  SLVIGSSTLFS      XRCC6   VYPEE   LVIGS      11              0.001822   
3173  PESKDDKHGSY       TCP1   LIKLH   ESKDD      11              0.004079   

      AP without flanks score HIT/DECOY  y  
0                    0.010553       HIT  1  
1                    0.489819       HIT  1  
2                    0.029527       HIT  1  
3                    0.130120       HIT  1  
4                    0.069398       HIT  1  
...                       ...       ... ..  
3169                 0.119909     DECOY  0  
3170                 0.153942     DECOY  0  
3171                 0.009945     DECOY  0  
3172                 0.003727     DECOY  0  
3173                 0.003733     DECOY  0  

[3174 rows x 9 columns]

In [8]:
#### performance of MHCflurry
fpr, tpr, _ = roc_curve(MAPP_df['y'],MAPP_df['AP without flanks score'])
auroc = auc(fpr, tpr)
print("auroc without flank: ", auroc)

precision, recall, _ = precision_recall_curve(MAPP_df['y'],MAPP_df['AP without flanks score'])
auprc = auc(recall, precision)
print("auprc without flank: ", auprc)

fpr, tpr, _ = roc_curve(MAPP_df['y'],MAPP_df['AP with flanks score'])
auroc = auc(fpr, tpr)
print("auroc with flank: ", auroc)

precision, recall, _ = precision_recall_curve(MAPP_df['y'],MAPP_df['AP with flanks score'])
auprc = auc(recall, precision)
print("auprc with flank: ", auprc)

auroc without flank:  0.5700788098234686
auprc without flank:  0.4014418235066789
auroc with flank:  0.579094051992807
auprc with flank:  0.4020565710724375
